## grab primary MERRA2 fields for the bounding box +/- 5 degrees, make distance and azimuth,  and write out 

To help us composite gridded quantities by distance from the CWV=55mm boundary contour, with a `groupby().mean()` operation, we need to construct a new distance field on the MERRA2 grid.

In [2]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import geopandas as gp

from shapely.geometry import Point
import pyproj
geodesic = pyproj.Geod(ellps='WGS84')

import warnings
warnings.filterwarnings('ignore')

# Open MERRA2 files and combine all the wanted fields 
### (2D hourly collections on goldsmr4) 

In [2]:
d2diag = xr.open_dataset('https://goldsmr4.gesdisc.eosdis.nasa.gov/dods/M2T1NXINT')
d2flux = xr.open_dataset('https://goldsmr4.gesdisc.eosdis.nasa.gov/dods/M2T1NXFLX')
d2slv = xr.open_dataset('https://goldsmr4.gesdisc.eosdis.nasa.gov/dods/M2T1NXSLV')

var_diag = ['dqvdt_phy','dqvdt_dyn','dqvdt_ana','uflxqv','vflxqv','swnettoa']
var_flux = ['eflux','evap','hflux','preccon','precanv','prectot','prevtot','prectotcorr']
var_slv = [ 't2m','ts','t850','t500','q850','q500','slp','tqv','tqi','tql',  \
           'disph','tropt' ] # dummy ones to re-use as distance and direction

d2super = d2diag[ var_diag ].merge(d2flux[ var_flux ]).merge(d2slv[ var_slv ])\
        .rename({"disph":"distance"}).rename({"tropt":"dir_from_centroid"})


In [22]:
#var_diag = ['dqvdt_phy','dqvdt_dyn','dqvdt_ana','uflxqv','vflxqv','swnettoa']
var_flux_min = ['prectot']
var_slv_min = ['tqv','disph','tropt'] # dummy ones to re-use as distance and direction

d2min = d2flux[ var_flux_min ].merge(d2slv[ var_slv_min ])\
        .rename({"disph":"distance"}).rename({"tropt":"dir_from_centroid"})

d2min

<xarray.Dataset>
Dimensions:            (time: 379848, lat: 361, lon: 576)
Coordinates:
  * time               (time) datetime64[ns] 1980-01-01T00:30:00 ... 2023-05-...
  * lat                (lat) float64 -90.0 -89.5 -89.0 -88.5 ... 89.0 89.5 90.0
  * lon                (lon) float64 -180.0 -179.4 -178.8 ... 178.1 178.8 179.4
Data variables:
    prectot            (time, lat, lon) float32 ...
    tqv                (time, lat, lon) float32 ...
    distance           (time, lat, lon) float32 ...
    dir_from_centroid  (time, lat, lon) float32 ...
Attributes:
    title:        MERRA-2 tavg1_2d_flx_Nx: 2d,1-Hourly,Time-Averaged,Single-L...
    Conventions:  COARDS\nGrADS
    dataType:     Grid
    history:      Wed May 24 22:31:24 GMT 2023 : imported by GrADS Data Serve...

## Read all lakes in 2014-2018 
#### lakes dataframe was "improved" in LakeCaseStudy.ipynb with firsttime added

In [3]:
df = pd.read_csv('ccvls_stats_2014-2018.improved.csv')
# df

In [4]:
equatorcases = df[ abs(df.coastlat) < 10 ] # 162 of them exceeding 1 day 
eq7cases = equatorcases[ equatorcases.dur_days > 6 ]  # 39 of them exceeding a week (more than 6 in days) 
eq7cases.maxarea.size

39

In [14]:
eq7cases

Unnamed: 0.1  Unnamed: 0             lasttime          duration  \
92              92         102  2018-11-20 11:00:00   7 days 00:00:00   
99              99         109  2018-11-16 21:00:00   7 days 01:00:00   
127            127         137  2018-10-22 15:00:00   8 days 01:00:00   
412            412         422  2018-03-19 12:00:00   7 days 18:00:00   
663            663         673  2017-12-01 11:00:00  11 days 00:00:00   
853            853         863  2017-05-28 11:00:00   7 days 04:00:00   
934            934         944  2017-05-09 01:00:00  10 days 03:00:00   
935            935         945  2017-05-08 05:00:00   9 days 07:00:00   
936            936         946  2017-05-06 20:00:00   7 days 22:00:00   
937            937         947  2017-05-06 20:00:00   7 days 22:00:00   
938            938         948  2017-05-06 18:00:00   7 days 20:00:00   
963            963         973  2017-04-29 11:00:00   9 days 15:00:00   
964            964         974  2017-04-29 11:00:00   9 days 15:00:00   
1121          1121        1131  2017-02-28 21:00:00  10 days 04:00:00   
1122          1122        1132  2017-02-27 22:00:00   9 days 05:00:00   
1219          1219        1229  2017-01-07 14:00:00   8 days 19:00:00   
1419          1419        1429  2016-05-02 05:00:00  10 days 10:00:00   
1420          1420        1430  2016-05-01 00:00:00   9 days 05:00:00   
1421          1421        1431  2016-04-29 17:00:00   7 days 22:00:00   
1462          1462        1472  2016-04-15 08:00:00  12 days 01:00:00   
1464          1464        1474  2016-04-10 23:00:00   7 days 16:00:00   
1936          1936        1946  2015-09-17 21:00:00  10 days 05:00:00   
2040          2040        2050  2015-05-07 03:00:00  14 days 15:00:00   
2041          2041        2051  2015-05-06 21:00:00  14 days 09:00:00   
2042          2042        2052  2015-05-05 21:00:00  13 days 09:00:00   
2043          2043        2053  2015-05-05 07:00:00  12 days 19:00:00   
2044          2044        2054  2015-05-05 01:00:00  12 days 13:00:00   
2045          2045        2055  2015-05-01 21:00:00   9 days 09:00:00   
2046          2046        2056  2015-05-01 17:00:00   9 days 05:00:00   
2047          2047        2057  2015-04-30 17:00:00   8 days 05:00:00   
2049          2049        2059  2015-04-30 02:00:00   7 days 14:00:00   
2050          2050        2060  2015-04-29 21:00:00   7 days 09:00:00   
2155          2155        2165  2015-03-21 17:00:00   7 days 22:00:00   
2156          2156        2166  2015-03-21 09:00:00   7 days 14:00:00   
2461          2461        2471  2014-05-11 18:00:00   9 days 09:00:00   
2462          2462        2472  2014-05-11 15:00:00   9 days 06:00:00   
2501          2501        2511  2014-04-19 22:00:00  12 days 11:00:00   
2504          2504        2514  2014-04-17 06:00:00   9 days 19:00:00   
2508          2508        2518  2014-04-12 21:00:00   7 days 04:00:00   

          areatime  tqv_values     maxarea                         filename  \
92     1892.647086        55.0   21.949572  2018_11_20_11_lat4p616S.geojson   
99     2667.686153        55.0   74.513991  2018_11_16_21_lat8p412S.geojson   
127    5261.462676        55.0  113.820949  2018_10_22_15_lat9p781N.geojson   
412   16255.056979        55.0  221.334618  2018_03_19_12_lat9p318S.geojson   
663    6534.053999        55.0   92.725686  2017_12_01_11_lat9p845S.geojson   
853    5690.601728        55.0   50.852737  2017_05_28_11_lat5p291S.geojson   
934   13759.444603        55.0  308.867756  2017_05_09_01_lat3p355S.geojson   
935    9695.018611        55.0  124.822389  2017_05_08_05_lat1p160N.geojson   
936    8426.735231        55.0  124.822389  2017_05_06_20_lat3p790S.geojson   
937    8629.762052        55.0  124.822389  2017_05_06_20_lat0p918N.geojson   
938    8419.365461        55.0  124.822389  2017_05_06_18_lat1p979S.geojson   
963    7493.295378        55.0   97.229094  2017_04_29_11_lat7p001S.geojson   
964    8972.043947        55.0  121.819141  2017_04_29_11_

In [24]:
# MIN CASE with distance and azimuth
d2loop = d2min

for icase in range(len(eq7cases)): 

# Continue from where it last crashed/ended:
#    if(icase < 8): 
#        continue

# Select case and move forward
    case = eq7cases.iloc[icase]
    filename = case.filename
    print(icase, ' case ',filename)

    gdf = gp.read_file('GEOJSONS/'+filename)
    gdf.set_crs(epsg = "4326", inplace = True)
    bounds = gdf.bounds
    bounds.minx.min(), bounds.maxx.max(), bounds.miny.min(), bounds.maxy.max(),

    d2 = d2loop.sel( lat =slice(bounds.miny.min()-5, bounds.maxy.max()+5),
                     lon =slice(bounds.minx.min()-5, bounds.maxx.max()+5), 
                     time=slice(gdf.time.min(),gdf.time.max())             )

    # Put a dummy distance field, initially large so min operator will replace it later 
    d2.distance.values = d2.distance.values*0 +999.
    d2['distance'].attrs['long_name'] = 'distance outside nearest lake (negative for interiors)'
    d2['distance'].attrs['units'] = 'degrees lat/lon'
    d2['dir_from_centroid'].attrs['long_name'] = 'navigation angle from centroid of nearest lake'
    d2['dir_from_centroid'].attrs['units'] = 'nav.degrees'

# Make an array of Points that are the gridpoints or MERRA2
    lat2d = d2.lat.values[:,None]   + d2.lon.values*0
    lon2d = d2.lat.values[:,None]*0 + d2.lon.values
    points = gp.GeoSeries( [Point(x, y) for x, y in zip(lon2d.ravel(),lat2d.ravel()) ] )\
            .set_crs(epsg = "4326", inplace = True)
    
    for i in range(gdf.time.size):     # the number of rows in the geoseries (polygons), not # of times
        # print(gdf.time[i])
# Compute distance and direction fields:
# DIST: must compute distance to polygon BOUNDARY to get nonzero values inside it 
        dist = points.distance(gdf.geometry[i].boundary).values.reshape(len(d2.lat),len(d2.lon))
        isin = points.within(gdf.geometry[i]).values.reshape(len(d2.lat),len(d2.lon))
        dist *= (-2)*(isin-0.5)  # make SIGNED distance from boundary, positive is exterior 

# DIR: use geodesic method imported above. 
# Direction to centroid of WHOLE LAKE (multi-polygon in general) at this time
        agg_lake_at_time = gdf.geometry[ np.where(gdf.time == gdf.time[i])[0] ].unary_union
        centlon = agg_lake_at_time.centroid.x
        centlat = agg_lake_at_time.centroid.y
    
    #Careful, use j not i in stupid explicit loop, since geodesic takes scalar only 
        dir_to = []
        for j in range(len(lon2d.ravel())): 
            fwd_az,back_az,d = geodesic.inv(lon2d.ravel()[j], lat2d.ravel()[j], [centlon], [centlat])
            dir_to.append(fwd_az+180)
        dir_to = np.array(dir_to).reshape(len(d2.lat),len(d2.lon))
    
# if dist is less than previous distance in the dataset, replace that previous
        oldd = d2.sel(time = gdf.time[i],method='nearest').distance
        mindist = np.minimum(dist,oldd)
    # closer = np.where(dist < oldd) # a spatial index of where, couldn't make it work
    
# Here's a tricky thing, assigning values to the dataset - need to use index instead of sel
# sel cannot be used for assignment of values, only extraction of them. 
# idx of the time in d2 corresponding to this geometry object in the gdf
        idx = d2.indexes["time"].get_indexer([gdf.time[i]],method='nearest')[0]

# Assign the new mindist field at this time, & dir where (dist < oldd) for this object
        d2["distance"][idx] = mindist
    
# Assigning direction is even trickier. I can't seem to use "closer" 
# to reassign direction only in places where dist < oldd, such that 
# the direction field would be from the centroid of the CLOSEST polgon. 
# Instead, let's just assign direction from the centroid of the whole 
# unary union of all the individual polygons 

        d2["dir_from_centroid"][idx] = dir_to

#  these 1-hour averages are 30 minutes offset from tqv, argh. 
# just manually spray the mindist values to the next earlier time level too 
        try: 
            d2["distance"][idx-1] = mindist
            d2["dir_from_centroid"][idx-1] = dir_to

        except: 
            print('end of loop issue (first time in time-backward sequence over geometries)')

            
    print('Writing outputs: '+filename)
    d2.to_netcdf('~/Box/VaporLakes/data/LAKEBYLAKE/MERRA2_2D/MINIMAL/'+\
        filename[0:-8]+'.M2_min2D_distance.nc')
    

0  case  2018_11_20_11_lat4p616S.geojson
Writing outputs: 2018_11_20_11_lat4p616S.geojson
1  case  2018_11_16_21_lat8p412S.geojson
Writing outputs: 2018_11_16_21_lat8p412S.geojson
2  case  2018_10_22_15_lat9p781N.geojson
Writing outputs: 2018_10_22_15_lat9p781N.geojson
3  case  2018_03_19_12_lat9p318S.geojson
Writing outputs: 2018_03_19_12_lat9p318S.geojson
4  case  2017_12_01_11_lat9p845S.geojson
Writing outputs: 2017_12_01_11_lat9p845S.geojson
5  case  2017_05_28_11_lat5p291S.geojson
Writing outputs: 2017_05_28_11_lat5p291S.geojson
6  case  2017_05_09_01_lat3p355S.geojson
Writing outputs: 2017_05_09_01_lat3p355S.geojson
7  case  2017_05_08_05_lat1p160N.geojson
Writing outputs: 2017_05_08_05_lat1p160N.geojson
8  case  2017_05_06_20_lat3p790S.geojson
Writing outputs: 2017_05_06_20_lat3p790S.geojson
9  case  2017_05_06_20_lat0p918N.geojson
Writing outputs: 2017_05_06_20_lat0p918N.geojson
10  case  2017_05_06_18_lat1p979S.geojson
Writing outputs: 2017_05_06_18_lat1p979S.geojson
11  case 

In [8]:
!ls ~/Box/VaporLakes/data/LAKEBYLAKE

2014_01_28_05_lat11p47S.IMERGcal_halfhourly.nc
2014_01_28_05_lat11p47S.IMERGhq_halfhourly.nc
2017-05-27.GFSleadtimes.PW.nc
2017_05_28_11_lat5p291S.M2_composite.nc
GridSat_IRimagery
IMERG
MERRA2_2D
direction_field_wrong_fixable_nowfixed_deleteme_after_checking


In [ ]:
!date

# Consider making IDV bundle for it 
### by relacing filename inside .xidv of a template bundle

In [21]:
!ls ../../Library/CloudStorage/Box-Box/VaporLakes/data/LAKEBYLAKE/

2017_05_06_20_lat3p790S.M2_all2Dfields.nc
2017_05_08_05_lat1p160N.M2_all2Dfields.nc
2017_05_09_01_lat3p355S.M2_all2Dfields.nc
2017_05_28_11_lat5p291S.IMERG_30minute.nc
2017_05_28_11_lat5p291S.M2_all2Dfields.nc
2017_05_28_11_lat5p291S.M2_all2Dfields_myfirst.nc
2017_05_28_11_lat5p291S.M2_composite.nc
2017_12_01_11_lat9p845S.M2_all2Dfields.nc
2018_03_19_12_lat9p318S.M2_all2Dfields.nc
2018_10_22_15_lat9p781N.M2_all2Dfields.nc
2018_11_16_21_lat8p412S.M2_all2Dfields.nc
2018_11_20_11_lat4p616S.M2_all2Dfields.nc
direction_field_wrong_fixable


In [32]:
# inplace replacement works 

!cp ../../Library/CloudStorage/Box-Box/VaporLakes/Landfalling_Lake_Event_Displays.xidv test.txt

!sed -i '' "s/2017_05_28_11_lat5p291S.M2_all2Dfields_myfirst/2018_11_20_11_lat4p616S.M2_all2Dfields.nc]/g" test.txt

In [33]:
!grep Box test.txt

                                <string><![CDATA[/Users/brianmapes/Library/CloudStorage/Box-Box/VaporLakes/data/LAKEBYLAKE/2018_11_20_11_lat4p616S.M2_all2Dfields.nc].nc]]></string>
                        <string><![CDATA[/Users/bem/Box/VaporLakes/data/LAKEBYLAKE/2018_11_20_11_lat4p616S.M2_all2Dfields.nc].nc]]></string>
                                        <string><![CDATA[/Users/bem/Box/VaporLakes/data/LAKEBYLAKE/2018_11_20_11_lat4p616S.M2_all2Dfields.nc].nc]]></string>


In [ ]:
template = '../../Library/CloudStorage/Box-Box/VaporLakes/Landfalling_Lake_Event_Displays.xidv'

In [17]:
gdf.time[i]

'2018-11-19T17:00:00'